# Meta Data Preparation
In this notebook, we read in the imaging maps, galaxy catalog, and the bright stars masks. Make healix masks, etc.

#### Read required modules

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import fitsio as ft
import pandas  as pd
import numpy   as np
import healpy  as hp
import seaborn as sns
import pymangle
from   glob import glob
import sys
from   utils import hpixsum, hpix2radec

## Cut objects based on Bright stars masks
#### Q1: Bright stars mask?

In [ ]:
sdss  = pymangle.Mangle('/Volumes/TimeMachine/data/bright_object_mask_rykoff_pix.ply')
tych1 = pymangle.Mangle('/Volumes/TimeMachine/data/tycho2mask-0Vmag10.pol')
tych2 = pymangle.Mangle('/Volumes/TimeMachine/data/tycho2mask-10Vmag11.pol')
tych3 = pymangle.Mangle('/Volumes/TimeMachine/data/tycho2mask-11Vmag115.pol')

ra,dec = hpix2radec(256, np.arange(12*256*256))
msdss = sdss.contains(ra,  dec)
mt1   = tych1.contains(ra, dec)
mt2   = tych2.contains(ra, dec)
mt3   = tych3.contains(ra, dec)

In [ ]:
mall = msdss | mt1 | mt2 | mt3
plt.scatter(ra[mall], dec[mall], 1.0, color='k', marker='.', alpha=0.2)
plt.scatter(ra[msdss], dec[msdss], 1.0, color='b', marker='.')

In [ ]:
ls /Volumes/TimeMachine/data/DR7/

In [ ]:
def apply_cut(cat_in, cat_out, cat_hp):
    data = ft.read(cat_in, lower=True)
    n = data.size
    print('original cat. size,', data.size)
    # 3760648
    # sdss = pymangle.Mangle('/Volumes/TimeMachine/data/bright_object_mask_rykoff_pix.ply')
    tych1= pymangle.Mangle('/Volumes/TimeMachine/data/tycho2mask-0Vmag10.pol')
    tych2= pymangle.Mangle('/Volumes/TimeMachine/data/tycho2mask-10Vmag11.pol')
    tych3= pymangle.Mangle('/Volumes/TimeMachine/data/tycho2mask-11Vmag115.pol')
    mask = ~data['brightstarinblob']
    # mask &= (data['anymask_g'] == 0) 
    # mask &= (data['anymask_r'] == 0) 
    # mask &= (data['anymask_z'] == 0) 
    data1 = data[mask]
    #print('data size after anymask & brightstarinblob cuts {:.4f} %'.format(data1.size/float(n)))
    print('data size after brightstarinblob cuts {:.4f} %'.format(data1.size/float(n)))
    mask_stars = np.zeros(data1.size, '?')
    for i,smask in enumerate([tych1, tych2, tych3]): #sdss, 
        mask_stars |= smask.contains(data1['ra'], data1['dec'])
        print(i)
    print('# of removed sources due to bright stars {}'.format(mask_stars.sum()))
    data2 = data1[~mask_stars]
    ft.write(cat_out, data2, clobber=True)
    #
    # pixelate
    hpm = hpixsum(256, data2['ra'], data2['dec'])
    hp.write_map(cat_hp, hpm, fits_IDL=False, overwrite=True)
    

apply_cut('/Volumes/TimeMachine/data/DR7/eBOSS.ELG.NGC.DR7.fits',
          '/Volumes/TimeMachine/data/DR7/eBOSS.ELG.NGC.DR7.cut.fits',
          '/Volumes/TimeMachine/data/DR7/eBOSS.ELG.NGC.DR7.cut.hp256.fits')

In [ ]:
ls /Volumes/TimeMachine/data/DR7/

## Combine the galaxy density and imaging maps 

In [ ]:
def G_to_C(mapi, res_in=1024, res_out=256):
    thph = hp.pix2ang(res_out, np.arange(12*res_out*res_out))
    r = hp.Rotator(coord=['C', 'G'])
    thphg = r(thph[0], thph[1])
    hpix  = hp.ang2pix(res_in, thphg[0], thphg[1])
    return mapi[hpix]

def extract_keys(mapi):
    band = mapi.split('/')[-1].split('_')[3]
    sysn = mapi.split('/')[-1].split('_')[6]
    oper = mapi.split('/')[-1].split('_')[-1].split('.')[0]
    return '_'.join((sysn, band, oper))

def IvarToDepth(ivar):
    """
        function to change IVAR to DEPTH
    """
    depth = nanomaggiesToMag(5./np.sqrt(ivar))
    return depth

def nanomaggiesToMag(nm):
    return -2.5 * (np.log10(nm) - 9.)

def maskmap(filename, nside=256):    
    data   = ft.read(filename, lower=True)
    if 'ivar' in filename:
        print('change ivar to depth ...')
        signal = IvarToDepth(data['signal'])
    elif 'fwhm' in filename:
        print('change fwhm to arcsec ...')
        signal = data['signal']*0.262
    else:
        signal = data['signal']
    #
    output = np.empty(12*nside*nside)
    output.fill(np.nan)
    output[data['pixel']] = signal
    return output

def combine_maps(maps, cat_hp):
    dr7meta = {}
    for mapi in maps:    
        namei  = extract_keys(mapi)    
        print('working on ... %s'%mapi.split('/')[-1])
        if 'ivar' in namei:namei = namei.replace('ivar', 'depth')
        dr7meta[namei] = maskmap(mapi)
    gaia = ft.read('/Volumes/TimeMachine/data/gaia/Gaia.dr2.bGT10.12g17.hp256.fits', lower=True)
    ebv  = hp.read_map('/Volumes/TimeMachine/data/healSFD_256_fullsky.fits', verbose=False)
    dr7meta['nstar'] = gaia['hpstardens'].astype('f8')
    dr7meta['ebv']   = ebv

    # read Lenz et. al. map
    # ebvhd   = ft.read('/Volumes/TimeMachine/data/ebv_lhd.hpx.fits', lower=True)['ebv'].astype('f8')
    # ebvhd_c = G_to_C(ebvhd)
    # nan     = np.isnan(ebvhd_c)
    # ebvhd_m = ebvhd_c.copy()
    # ebvhd_c[nan] = ebv[nan]-.5

    # H II map
    hii = ft.FITS('/Volumes/TimeMachine/data/NHI_HPX.fits', lower=True)
    Hii = hii[1].read()
    neg_mask = (Hii['nhi']<=0.0)
    Hiic = G_to_C(Hii['nhi'])
    Hineg = np.argwhere(Hiic<=0.0).flatten()
    neighbors = hp.get_all_neighbours(256, Hineg)
    Hiic[Hineg] = np.mean(Hiic[neighbors], axis=0) # fill in negative pixels
    #
    #dr7meta['ebv_lenz']      = ebvhd_c
    #dr7meta['ebv_lenz_org']  = ebvhd_m
    #dr7meta['debv_lenz_sfd'] = ebvhd_c-ebv
    dr7meta['logHI'] = np.log(Hiic)
    DR7meta = pd.DataFrame(dr7meta)
    DR7meta['count_min'] = DR7meta[['_'.join(('count',b, 'fracdet')) for b in 'rgz']].min(axis=1) # add min of counts
    #
    # ngal
    hpgal    = hp.read_map(cat_hp, verbose=False)
    DR7meta['ngal'] = hpgal
    #
    # mask
    nanids = np.unique(np.where(DR7meta.isna())[0])
    nonan  = np.ones(12*256*256, '?')
    nonan[nanids] = False
    mask   = (DR7meta['depth_g_total'] >= 22.0) & (DR7meta['depth_z_total']>= 20.5)\
            &(DR7meta['depth_r_total'] >= 21.4) & (DR7meta['count_min']>= 0.2) & nonan
    DR7meta['mask'] = mask
    if len(np.where(DR7meta[DR7meta['mask']].isna())[0]) != 0:print('There is NaNs')
    avg = DR7meta['ngal'].sum()/DR7meta['count_min'].sum()
    # replace inf with nan
    # replace inf with nan
    DR7meta.replace([np.inf, -np.inf], np.nan, inplace=True)
    return DR7meta

In [ ]:
maps = glob('/Volumes/TimeMachine/data/DR7/sysmaps/DECaLS_DR7/nside256_oversamp4/DECaLS_DR7_band_*')
maps[0], len(maps)

In [ ]:
DR7meta = combine_maps(maps, '/Volumes/TimeMachine/data/DR7/eBOSS.ELG.NGC.DR7.cut.hp256.fits')

In [ ]:
DR7meta.to_hdf('/Volumes/TimeMachine/data/DR7/DR7meta.h5', 'DR7meta', mode='w', format='fixed')

In [ ]:
fig, ax = plt.subplots(nrows=10, ncols=3, figsize=(15, 32))
ax=ax.flatten()
for i,name in enumerate(DR7meta.columns):
    plt.sca(ax[i])
    hp.mollview(DR7meta[name], hold=True, title=name)

## Read the metadata and write out as a FITS table

In [ ]:
def hd5_2_fits(metaname, fitname, hpmask, hpfrac, cols):    
    metadata = pd.read_hdf(metaname)
    features = metadata[cols][metadata['mask']].values    
    hpind    = np.argwhere(metadata['mask']).flatten()
    nbar     = metadata['ngal'][metadata['mask']].sum() / metadata['count_min'][metadata['mask']].sum()
    label    = (metadata['ngal'][metadata['mask']] / metadata['count_min'][metadata['mask']])/nbar
    fracgood = metadata['count_min'][metadata['mask']]  
    
    # for n in metadata.columns:
    #     print('%20s : %d'%(n, np.isnan(metadata[metadata['mask']][n]).sum()))
    outdata = np.zeros(features.shape[0], 
                       dtype=[('label', 'f8'),
                              ('hpind','i8'), 
                              ('features',('f8', features.shape[1])),
                              ('fracgood','f8')])
    outdata['label']    = label
    outdata['hpind']    = hpind
    outdata['features'] = features
    outdata['fracgood'] = fracgood
    #
    #
    ft.write(fitname, outdata, clobber=True)
    print('Average N : %.8f'%nbar)
    #
    # 
    mask = np.zeros(12*256*256, '?')
    mask[hpind] = True
    hp.write_map(hpmask, mask, overwrite=True, fits_IDL=False)
    #
    #
    frac = np.zeros(12*256*256)
    frac[hpind] = fracgood
    hp.write_map(hpfrac, frac, overwrite=True, fits_IDL=False)

In [ ]:
cols   = ['ebv','logHI', 'nstar',\
          'depth_r_total', 'depth_g_total', 'depth_z_total',\
         'fwhm_r_mean', 'fwhm_g_mean', 'fwhm_z_mean',\
         'ccdskymag_r_mean', 'ccdskymag_g_mean', 'ccdskymag_z_mean',
         'exptime_r_total', 'exptime_g_total', 'exptime_z_total',
         'mjd_r_min', 'mjd_g_min', 'mjd_z_min'] 
fitname = '/Volumes/TimeMachine/data/DR7/eBOSS.ELG.NGC.DR7.table.fits'
metaname = '/Volumes/TimeMachine/data/DR7/DR7meta.h5'
hpmask   = '/Volumes/TimeMachine/data/DR7/mask.hp.256.fits'
hpfrac   = '/Volumes/TimeMachine/data/DR7/frac.hp.256.fits'

# call the function
hd5_2_fits(metaname, fitname, hpmask, hpfrac, cols)

## Split the FITS table into Training, Test and Validation sets

In [ ]:
from utils import read_split_write

In [ ]:
read_split_write('/Volumes/TimeMachine/data/DR7/eBOSS.ELG.NGC.DR7.table.fits',
                 '/Volumes/TimeMachine/data/DR7/eBOSS.ELG.NGC.DR7.table.5.r.npy',
                 k=5,
                 random=True)

In [ ]:
ls -lt /Volumes/TimeMachine/data/DR7/

# Split the FITS table for the mock footprint for testing

In [ ]:
dt = ft.read('/Volumes/TimeMachine/data/DR7/eBOSS.ELG.NGC.DR7.table.fits')
dt.dtype.names

In [ ]:
mock = hp.read_map('/Volumes/TimeMachine/data/mocks/mask.hp.256.fits') > 0

In [ ]:
data_on_mock = np.in1d(dt['hpind'], np.argwhere(mock).flatten())
data_on_mock.mean()

In [ ]:
dt_mock = dt[data_on_mock]
dt_mock.size

In [ ]:
from utils import split2Kfolds

In [ ]:
dt_mock5 = split2Kfolds(dt_mock)

In [ ]:
np.save('/Volumes/TimeMachine/data/DR7/eBOSS.ELG.NGC.DR7.mocks.table.5.r.npy', dt_mock5)